## 1. 라이브러리 및 데이터 준비

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

- 아래는 nn.lstm의 output, hidden state에 대해 설명함.  
  (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)  
- seq2seq 구현 참고 자료:  
  (https://codlingual.tistory.com/91)

In [2]:
# 학습 데이터
in_text = "nice to meet you"
out_text = "ravi de vous rencontrer"
#de_input = "<sos> ravi de vous rencontrer"
#out_target = "ravi de vous rencontrer <eos>"
# 문자 집합 생성
chars = set(in_text.split())
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}


In [3]:
outs = set(out_text.split()) 
out_to_index = {char: i+1 for i, char in enumerate(outs)}
out_to_index['<sos>'] = 0
out_to_index['<eos>'] = 5

index_to_out = {i: key for key, i in out_to_index.items()}


In [4]:
out_to_index

{'ravi': 1, 'vous': 2, 'rencontrer': 3, 'de': 4, '<sos>': 0, '<eos>': 5}

In [5]:
index_to_out

{1: 'ravi', 2: 'vous', 3: 'rencontrer', 4: 'de', 0: '<sos>', 5: '<eos>'}

In [6]:
# 하이퍼파라미터
sequence_length = 4  # 입력 길이
vocab_size = len(chars)  # 문자 집합 크기
embedding_dim = 10  # 임베딩 차원
hidden_size = 128  # LSTM 은닉 상태 크기
num_layers = 1  # LSTM 레이어 개수

In [7]:
for key, value in index_to_char.items():
    print(key,value)

0 to
1 meet
2 nice
3 you


## 2. 데이터 전처리 (One-Hot Encoding)

In [8]:
# One-Hot Encoding 함수
def one_hot_encode(indices, vocab_size):
    one_hot = np.zeros((len(indices), vocab_size), dtype=np.float32)
    for i, idx in enumerate(indices):
        one_hot[i, idx] = 1.0
    return one_hot

In [9]:
# 입력과 출력 데이터 생성
input_sequences = []

for i, _ in index_to_char.items():
    input_sequences.append(i)

In [10]:
input_sequences

[0, 1, 2, 3]

In [11]:
out1 = ('<sos> ' + out_text).split()
out2 = (out_text + ' <eos>').split()

In [12]:
out2

['ravi', 'de', 'vous', 'rencontrer', '<eos>']

In [13]:
#input_decode = []
input_decode = [out_to_index[x] for x in out1]
target = [out_to_index[x] for x in out2]  

In [14]:
x_1 = one_hot_encode(input_sequences,4)
x_2 = one_hot_encode(input_decode,6)
x = x_1,x_2
y = one_hot_encode(target,6)  

In [15]:
class TransDataset(Dataset):
    def __init__(self, x,y):
        self.x0 = x[0]
        self.x1 = x[1]
        self.y  = y

    def __len__(self):
        return 1

    def __getitem__(self, index):
        return (self.x0, self.x1), self.y
        

In [16]:
dataset = TransDataset(x,y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
print('the number of images', len(dataset))  
#print("----batch size",options.batchSize)

the number of images 1


In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim=4, hid_dim=6):
        super().__init__()
        self.hid_dim = hid_dim
        self.rnn = nn.LSTM(input_dim, hid_dim)
        
    def forward(self, src):
        
        # embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(src)
        
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        return hidden, cell


In [18]:
class Decoder(nn.Module):
    def __init__(self, output_dim=6, hid_dim=6):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        
        self.rnn = nn.LSTM(output_dim, hid_dim)
        
    def forward(self, src, hidden, cell):
        
        
        output, (hidden, cell) = self.rnn(src, (hidden, cell))
        
        
        return output, hidden, cell

In [19]:
class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder):
       super().__init__()
       
       self.encoder = encoder
       self.decoder = decoder
       #self.device = device
       #print(f"output_dim : {self.decoder.output_dim}")
       # Encoder와 Decoder의 hidden dim이 같아야 함
       assert encoder.hid_dim == decoder.hid_dim

       
   def forward(self, src, trg, teacher_force=True):
       # src = [src len, batch size]
       # trg = [trg len, batch size]
       
       trg_len = trg.shape[0]
       batch_size = trg.shape[1]
       trg_vocab_size = 6#self.decoder.ouput_dim
                        #self.decoder
       
       # decoder 결과를 저장할 텐서
       outputs = torch.zeros(trg_len, batch_size, trg_vocab_size)
       
       # Encoder의 마지막 은닉 상태가 Decoder의 초기 은닉상태로 쓰임
       hidden, cell = self.encoder(src)
       
       # Decoder에 들어갈 첫 input은 <sos> 토큰
       input = trg[0, :]
       #print(f"input.shape : {input.shape}")
       
       # target length만큼 반복
       # range(0,trg_len)이 아니라 range(1,trg_len)인 이유 : 0번째 trg는 항상 <sos>라서 그에 대한 output도 항상 0 
       for t in range(1, trg_len):
           
           output, hidden, cell = self.decoder(input.unsqueeze(0), hidden, cell)
           outputs[t] = output
           
           
           # 확률 가장 높게 예측한 토큰
           top1 = output.argmax(1) 
           
           # techer_force = 1 = True이면 trg[t]를 아니면 top1을 input으로 사용
           input = trg[t] if teacher_force else top1
       
       return outputs

In [20]:
encoder = Encoder()
decoder = Decoder(6,6)
model = Seq2Seq(encoder, decoder)

In [21]:
model.decoder.output_dim

6

In [22]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [23]:
x_1.shape

(4, 4)

In [ ]:
model.train()

for epoch in range(5000):
    data_iterator = tqdm(dataloader, total=1, ncols=150, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    for sampleIndex, sample in enumerate(data_iterator):  
        optimizer.zero_grad()
        x_0 = sample[0][0].permute(1,0,2)
        x_1 = sample[0][1].permute(1,0,2)
        y = sample[1].permute(1,0,2)
        out_put = model(x_0,x_1)
        out_put = out_put.view(-1, 6)
        y = y.view(-1,6)
        loss = criterion(out_put, y)
            
        loss.backward()
        optimizer.step()
        status = str(epoch + 1) + ' loss: '
        status += '%0.4f '%loss.data.item()
        data_iterator.set_description(status)
    #print(sampleIndex, x_0.shape, x_1.shape, y.shape)

1708 loss: 0.8655 : 100%|██████████| 1/1 [00:00<00:00, 222.77it/s]                                                                                    


In [25]:
out_put.shape[-1]

6

In [26]:
y.shape

torch.Size([5, 6])